# Stack - Engenharia de Dados do Zero

- **Task**
  - Converta os arquivos em csv para parquet e os envie para processing zone.

- **Dataset**
  - Usaremos esse dataset https://www.kaggle.com/nhs/general-practice-prescribing-data

### Modos de leitura
- **permissive**: *Define todos os campos para NULL quando encontra registros corrompidos e coloca todos registros corrompidos em uma coluna chamada _corrupt_record.* (default)

- **dropMalformed**: *Apaga uma linha corrompida ou que este não consiga ler.*

- **failFast**: *Falha imediatamente quando encontra uma linha que não consiga ler.*

In [ ]:
# ler arquivos vários arquivos csv do dbfs com spark
# Lendo todos os arquivos .csv do diretório bigdata (>4GB)

df = spark.read.format("csv")\
.option("header", "True")\
.option("inferSchema","True")\
.load("/FileStore/tables/bigdata/*.csv")

In [ ]:
df.printSchema()

In [ ]:
# imprime as 10 primeiras linhas do dataframe
display(df.head(10))

In [ ]:
# conta a quantidade de linhas
df.count()

#### Leva os dados convertidos para a Processing Zone

- *Atente para NÃO escrever e ler arquivos parquet em versoes diferentes*

In [ ]:
# Converte para formato parquet
df.write.format("parquet")\
.mode("overwrite")\
.save("/FileStore/tables/processing/df-parquet-file.parquet")

In [ ]:
# lendo arquivos parquet
# atente para a velocidade de leitura

df_parquet = spark.read.format("parquet")\
.load("/FileStore/tables/processing/df-parquet-file.parquet")

In [ ]:
# conta a quantidade de linhas do dataframe
df_parquet.count()

In [ ]:
%scala
// script para pegar tamanho em Gigabytes
val path="/FileStore/tables/processing/df-parquet-file.parquet"
val filelist=dbutils.fs.ls(path)
val df_temp = filelist.toDF()
df_temp.createOrReplaceTempView("adlsSize")


In [ ]:
%sql
-- consulta a view criada.
select round(sum(size)/(1024*1024*1024),3) as sizeInGB from adlsSize

In [ ]:
display(df_parquet.head(10))

In [ ]:
#Add columns to DataFrame using SQL
df_parquet.createOrReplaceTempView("view_df_parquet")

spark.sql("SELECT BNF_CODE as Bnf_code \
                  ,SUM(ACT_COST) as Soma_Act_cost \
                  ,SUM(QUANTITY) as Soma_Quantity \
                  ,SUM(ITEMS) as Soma_items \
                  ,SUM(ACT_COST) as Media_Act_cost \
           FROM view_df_parquet \
           GROUP BY bnf_code").show()